In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import tflearn as tfl
import skimage as ski
from sklearn import preprocessing

In [2]:
%matplotlib inline

INPUT_FOLDER = 'input/'
OUTPUT_FOLDER = 'output/'
TRAIN_IN_FOLDER = INPUT_FOLDER + 'train/'
TRAIN_OUT_FOLDER = OUTPUT_FOLDER + 'train/'
TYPES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
TYPES = ['BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK']
# TYPES = ['DOL', 'LAG']
SIZE = 128

# Helpers

In [3]:
def benchmark(func):
    def wrapper(*arg, **kw):
        t1 = time.process_time()
        res = func(*arg, **kw)
        t2 = time.process_time()
        return (t2 - t1), res, func.__name__
    return wrapper

def plot_image(img):
    plt.imshow(img)
    
def log(label, text):
    print(str(label) + ': ' + str(text))
    
@benchmark
def images(func, folder):
    for t in TYPES:
        path = folder + t
        files = os.listdir(path)
        print('==> ' + func.__name__ + ': ' + t + ' ' + str(len(files)) + ' files')
        for filename in files:
            func(filename, t)

def process_image(filename, code, debug=True):
    path = code + '/' + filename
    # print('==> Processing ' + TRAIN_IN_FOLDER + path)
    img = cv2.imread(TRAIN_IN_FOLDER + path)
    img = cv2.resize(img, (SIZE, SIZE), cv2.INTER_LINEAR)
    os.makedirs(TRAIN_OUT_FOLDER + code, exist_ok=True)
    cv2.imwrite(TRAIN_OUT_FOLDER + path, img);

# Feature extraction

In [188]:
images(process_image, TRAIN_IN_FOLDER)

==> process_image: BET 200 files
==> process_image: DOL 117 files
==> process_image: LAG 67 files
==> process_image: NoF 465 files
==> process_image: OTHER 299 files
==> process_image: SHARK 176 files


(64.33212399999684, None, 'images')

# Feature Selection

In [4]:
X = []
y = []

def read_image(filename, code):
    path = code + '/' + filename
    img = cv2.imread(TRAIN_OUT_FOLDER + path)
    X.append(img)
    y.append(code)
    
images(read_image, TRAIN_OUT_FOLDER)

X = np.array(X)
y = preprocessing.LabelEncoder().fit_transform(np.array(y))

log('Features', X.shape)
log('Labels', y.shape)


==> read_image: BET 200 files
==> read_image: DOL 117 files
==> read_image: LAG 67 files
==> read_image: NoF 465 files
==> read_image: OTHER 299 files
==> read_image: SHARK 176 files
Features: (1324, 128, 128, 3)
Labels: (1324,)


# Models

In [179]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import utils

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(len(TYPES), activation='softmax')(x)

# this is the model we will train
inceptionv3 = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
inceptionv3.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
Y = utils.to_categorical(y)
inceptionv3.fit(X, Y, validation_split=0.1)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
# for i, layer in enumerate(base_model.layers):
#   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in inceptionv3.layers[:172]:
   layer.trainable = False
for layer in inceptionv3.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
inceptionv3.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
inceptionv3.fit(X, Y, validation_split=0.2)

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Train on 1191 samples, validate on 133 samples
Epoch 1/1
1191/1191 [==============================] - 578s - loss: 1.6894 - val_loss: 5.0807

In [177]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

img_path = 'output/train/ALB/img_00003.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
log('X', x.shape)


resnet50 = ResNet50(weights='imagenet')
preds = resnet50.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

X: (1, 224, 224, 3)
Predicted: [('n04522168', 'vase', 0.19095436), ('n03637318', 'lampshade', 0.083283588), ('n03388043', 'fountain', 0.052196849)]


In [183]:
from keras.applications.inception_v3 import preprocess_input, decode_predictions

img_path = 'output/train/SHARK/img_00365.jpg'
img = image.load_img(img_path, target_size=(SIZE, SIZE))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)
log('X', x.shape)

preds = inceptionv3.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
log('Predicted:', preds)
log('Predicted:', TYPES[preds.argmax()])

X: (1, 299, 299, 3)
Predicted:: [[ 0.13179334  0.07251851  0.04496516  0.66650552  0.07442341  0.00979402]]
Predicted:: NoF


In [8]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()

model.add(Dense(10, activation='relu', input_shape=(SIZE, SIZE, 3)))
model.add(Dense(10, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
          
Y = keras.utils.to_categorical(y)
model.fit(X, Y, validation_split=0.1, epochs=5)
model.save('fisheries.h5')

Train on 1191 samples, validate on 133 samples
Epoch 1/5
1191/1191 [==============================] - 11s - loss: 13.1701 - acc: 0.1805 - val_loss: 16.1181 - val_acc: 0.0000e+00

In [7]:
import vis
from vis.visualization import visualize_cam
from keras.preprocessing import image

img_path = 'output/train/DOL/img_00165.jpg'
img = image.load_img(img_path, target_size=(SIZE, SIZE))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
log('X', x.shape)

preds = model.predict(x)
pred_class = preds.argmax()
log('Predicted:', preds)
log('Predicted:', TYPES[pred_class])

heatmap = visualize_cam(model, 12, [pred_class], img)
plt.imshow(heatmap)

X: (1, 128, 128, 3)
Predicted:: [[ 1.  0.  0.  0.  0.  0.]]
Predicted:: BET
Working on filters: [0]


AttributeError: module 'itertools' has no attribute 'izip'

# Submission

In [17]:
sample = pd.read_csv('input/sample_submission_stg1.csv')
print(sample.shape)

submission = pd.DataFrame(columns=['image','ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT'])
submission.image = sample.image
submission.ALB = np.random.rand(sample.shape[0])
submission.BET = np.random.rand(sample.shape[0])
submission.DOL = np.random.rand(sample.shape[0])
submission.LAG = np.random.rand(sample.shape[0])
submission.NoF = np.random.rand(sample.shape[0])
submission.OTHER = np.random.rand(sample.shape[0])
submission.SHARK = np.random.rand(sample.shape[0])
submission.YFT = np.random.rand(sample.shape[0])
submission.head()

# submission.to_csv('submission.csv', index=False)

(1000, 9)


,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.515140,0.866245,0.598021,0.996599,0.139812,0.004797,0.765467,0.600272
1,img_00007.jpg,0.640867,0.287336,0.103477,0.720666,0.362753,0.776995,0.013971,0.896490
2,img_00009.jpg,0.917814,0.175919,0.589010,0.811196,0.647969,0.730503,0.429205,0.153039
3,img_00018.jpg,0.801091,0.594597,0.347978,0.982384,0.208814,0.640148,0.017344,0.157358
4,img_00027.jpg,0.396065,0.339609,0.179840,0.254583,0.884746,0.427801,0.762989,0.978957


# References

* https://deepsense.io/deep-learning-right-whale-recognition-kaggle/
* http://matthewearl.github.io/2016/05/06/cnn-anpr/
* http://cs231n.stanford.edu/reports/2016/pdfs/285_Report.pdf
* https://www.kaggle.com/narae78/the-nature-conservancy-fisheries-monitoring/fish-detection/run/461339
